# Filtrado Colaborativo: *Bayesian Non-negative Matrix Factorization*

El modelo NMF supone un gran avance para la justificación de las predicciones proporcionas por el sistema a la vez que mantiene los buenos resultados presentados por el modelo PMF. Sin embargo, a pesar de eliminar los valores negativos en los factores, las predicciones siguen siendo difilies de explicar.

¿En qué rango se mueven los factores? ¿Que interpretación tiene cada valor numérico? ¿Qué un factor valga 0.8 quiere decir que es el doble de importante que uno que valga 0.4? Todas estas preguntas plantean dudas al usuario a la hora de entender qué esta haciendo el modelo y, por tanto, de confiar en los resultados que proporciona.

La explicación de recomendaciones en modelos de factorización matricial pasa por dar un significado probabilístico a los valores de los factores latentes. Si se logra que todos los factores tomen valores en el rango \[0, 1], será posible justificar las recomendaciones. De este modo, si un usuario tiene el valor 0.5 en el factor 2, querrá decir que existe una probabilidad de 0.5 de que al usuario le guste un item relacionado con el factor 2.

El modelo *BNMF (**Bayesian Non-negative Matrix Factorization**)* se basa en el principio de la factorización matricial, pero aporta un significado probabilístico a los factores. A pesar de estar basado en la misma idea, el modelo de BNMF difiere bastante de PMF y NMF. BNMF se basa en el modelo bayesiano que puede observarse en la siguiente imagen:


<img src="https://drive.google.com/uc?export=view&id=1ATFFIKDGGL1TJntfjiRoFrh7Mkwrl3pE" alt="Datos empleados por los sistemas de recomendación">

Donde:

- $\alpha$ representa un parámetro del modelo que simboliza la probabilidad de obtener solapamiento entre los diferentes *clusters* que forma el subespacio de factores latentes.

- $\beta$ representa un parámetro del modelo relacionado con el número de evidencias necesarias para formar parte de un *cluster*.

- $\kappa_{i,k}$ es una variable aleatoria de una distribución Beta. Esta variable representa la probabilidad de que a un usuario del grupo $k$ le guste el item $i$.

- $\vec{\theta}_u$ es un vector *k* dimensional de la variable aleatoria de una distribución Dirichlet. Este vector simboliza la probabilidad de que un usuario pertenezca a un grupo. Se emplea la distribución de Dirichlet porque conjuga perfectamente con la distribución de la variable categórica.

- $z_{u,i}$ es una variable aleatoria de la distribución categórica. Representa que el usuario $u$ votará el item $i$ si el usuario pertenece al grupo *k*.

- $\rho_{u,i}$ es una variable aleatoria de la distribución Binomial. La distribución Binomial permite modelar los votos discretos existente en la mayoría de los sistemas de recomendación.

Más allá de las matemáticas, debemos comprender que este modelo representa mejor un sistema de recomendación que otras implementaciones de la factorización matricial. La mayoría de modelos asumen una distribución normal de las votaciones al optimizar una función en base al error cuadrático medio. Por contra, BNMF asume que los votos de los usuarios son discretos y finitos, tal y como sucede cuando realizamos votaciones en una escala de 1 a 5 estrellas.


## Entrenamiento del modelo

Debido a las características bayesianas del modelo, el proceso de entrenamiento no puede realizarse mediante la técnica del descenso del gradiente. En su lugar se emplea inferencia variacional para obtener el conjunto de ecuaciones necesarias para ajustar los valores de los factores a los datos de entrenamiento.

Los factores de los usuarios se representan mediante el carácter $a$ y se calculan de acuerdo con la siguiente ecuación:

$$a_{u,k}=\frac {\gamma_{u,k}} {\sum_{f=1..K} \gamma_{u,f}}$$

Donde $\gamma_{u,k}$ se calcula:

$$\gamma_{u,k} = \alpha + \sum_{\{i | r_{u,i} \neq \bullet \}} \lambda_{u,i,k}$$

Los factores de los usuarios se representan mediante el carácter $b$ y se calculan de acuerdo con la siguiente ecuación:

$$b_{k,i} = \frac{\epsilon_{i,k}^+}{\epsilon_{i,k}^+ + \epsilon_{i,k}^-}$$

Donde $\epsilon_{i,k}^+$ y $\epsilon_{i,k}^-$ se calculan:

$$\epsilon_{i,k}^+ = \beta + \sum_{\{i | r_{u,i} \neq \bullet \}} \lambda_{u,i,k} \cdot r_{u,i}^+$$

$$\epsilon_{i,k}^- = \beta + \sum_{\{i | r_{u,i} \neq \bullet \}} \lambda_{u,i,k} \cdot r_{u,i}^-$$

Verificándose que:

$$r_{u,i}^+ = R \cdot r_{u,i}^*$$

$$r_{u,i}^- = R \cdot (1 - r_{u,i}^*)$$

Donde $r_{u,i}^*$ es el voto del usuario $u$ al item $i$ normalizado.

Todas las anteriores ecuaciones dependen del valor de $\lambda_{u,i,k}$, el cual puede calcularse como:

$$\lambda_{u,i,k} = \frac {\lambda_{u,i,k}^\prime} {\lambda_{u,i,1}^\prime + \cdots + \lambda_{u,i,K}^\prime}$$

$$\lambda_{u,i,k}^\prime = exp( \psi(\gamma_{u,k}) + r_{u,i}^+ \cdot \psi(\epsilon_{i,k}^+) +  r_{u,i}^- \cdot \psi(\epsilon_{i,k}^-) - R \cdot \psi(\epsilon_{i,k}^+ + \epsilon_{i,k}^-))$$

Siendo $\psi$ la función diggama definida como la derivada logarítmica de la función gamma.

Dadas estas ecuaciones, el algoritmo de entrenamiento quedaría del siguiente modo:

- Iniciar aleatoriamente los valores de $\gamma_{u,k}$
- Iniciar aleatoriamente los valores de $\epsilon_{i,k}^+$
- Iniciar aleatoriamente los valores de $\epsilon_{i,k}^-$
- Iterar hasta la convergencia:
    - Para cada usuario $u$:
        - Para cada item $i$ votado por el usuario $u$:
            - Para cada factor $k$:
                - Actualizar $\lambda_{u,k,i}$
    - Para cada usuario $u$:
        - Para cada factor $k$:
            - Actualizar $\gamma_{u,k}$
        - Para cada item $i$ votado por el usuario $u$:
            - Para cada factor $k$:
                - Actualizar $\epsilon_{i,k}^+$
                - Actualizar $\epsilon_{i,k}^-$
- Calcular $a_{u,k}$
- Calcular $b_{i,k}$

Una vez entrenando el modelo la predicción del voto del usuario $u$ al item $i$ puede calcularse como:

$$\hat{r}_{u,i} = \sum_{k=1..K} a_{u,k} \cdot b_{i,k}$$

Téngase en cuenta que esta predicción estará normalizada.

Veamos cómo hacer esto con código.


## Carga del dataset

Para ilustrar mejor el funcionamiento el algoritmo BNMF, vamos a desarrollar una implementación del mismo.

Para ello usaremos el dataset de [MovieLens 100K](https://grouplens.org/datasets/movielens/) que contiene 100.000 votos de 943 usuarios sobre 1682 películas. Este dataset ha sido dividido en votaciones de entrenamiento (80%) y votaciones de test (20%). Además, los códigos de usuarios e items, han sido modificados para que comience en 0 y terminen en el número de (usuarios / items) - 1.


Inicialmente definimos algunas constantes que nos serán necesarias durante la codificación del algoritmo:

In [0]:
import urllib
import random
from scipy.special import digamma
import scipy.special
import math

In [0]:
NUM_USERS = 943
NUM_ITEMS = 1682

MIN_RATING = 1
MAX_RATING = 5

Y cargamos el dataset:

In [0]:
ratings = [[None for _ in range(NUM_ITEMS)] for _ in range(NUM_USERS)] 

training_file = urllib.request.urlopen("https://drive.google.com/uc?export=view&id=1S4-sxOEvA3MDivaGf7iFirWqt1H6VtaH")
for line in training_file:
  [u, i, rating] = line.decode("utf-8").split("::")
  ratings[int(u)][int(i)] = int(rating)

Del mismo modo, cargamos la matriz de votaciones de test:

In [0]:
test_ratings = [[None for _ in range(NUM_ITEMS)] for _ in range(NUM_USERS)] 

test_file = urllib.request.urlopen("https://drive.google.com/uc?export=view&id=1LBgTF57DD2NA-petq_FaC1V-h7nrYIh9")
for line in test_file:
  [u, i, rating] = line.decode("utf-8").split("::")
  test_ratings[int(u)][int(i)] = int(rating)

## Entrenamiento del modelo

Veamos cómo podemos aplicar las fórmulas anteriores para entrenar el modelo.


Primero, definimos los parámetros del modelo:

In [0]:
NUM_FACTORS = 5
ALPHA = 0.8
BETA = 5
R = 4

Inicializamos las matrices $\gamma$, $\epsilon^+$ y $\epsilon^-$ necesarias para el cálculo de los factores con valores uniformes aleatorios en el intervalo \[0, 1].

In [0]:
gamma = [[random.random() for _ in range(NUM_FACTORS)] for _ in range(NUM_USERS)] 
ep = [[random.random() for _ in range(NUM_FACTORS)] for _ in range(NUM_ITEMS)] 
em = [[random.random() for _ in range(NUM_FACTORS)] for _ in range(NUM_ITEMS)] 

Vamos a hacer que el modelo aprenda. Ejecutamos tantas veces como iteraciones haya la actualización de los factores. Para ello, recorremos el conjunto de votos y vamos haciendo las actualizaciones correspondientes.

In [0]:
NUM_ITERATIONS = 5

In [0]:
def lambda_prime(gamma_u_k, r, ep_i_k, em_i_k):
    return math.exp(
        digamma(gamma_u_k) +
        (R*r * digamma(ep_i_k)) +
        (R*(1 - r) * digamma(em_i_k)) -
        (R * digamma(ep_i_k + em_i_k))
    )

In [14]:
for it in range(NUM_ITERATIONS):
    print("Iteración " + str(it + 1) + " de " + str(NUM_ITERATIONS))
  
    # Calculamos lambda
  
    lmbda = [[[0 for _ in range(NUM_FACTORS)] for _ in range(NUM_ITEMS)] for _ in range(NUM_USERS)]
    
    for u in range(NUM_USERS):
        for i in range(NUM_ITEMS):
            if ratings[u][i] != None:

                r = (ratings[u][i] - MIN_RATING) / (MAX_RATING - MIN_RATING)

                summation = 0

                for k in range(NUM_FACTORS):
                    lmbda[u][i][k] = lambda_prime(gamma[u][k], r, ep[i][k], em[i][k])

                    summation += lmbda[u][i][k]

                for k in range(NUM_FACTORS):
                    lmbda[u][i][k] /= summation
  
    # Reinicioamos gamma, epsilon+ y epsilon-
  
    gamma = [[ALPHA for _ in range(NUM_FACTORS)] for _ in range(NUM_USERS)] 
    ep = [[BETA for _ in range(NUM_FACTORS)] for _ in range(NUM_ITEMS)] 
    em = [[BETA for _ in range(NUM_FACTORS)] for _ in range(NUM_ITEMS)]  
  
  
    # Actualizamos
  
    for u in range(NUM_USERS):
        for i in range(NUM_ITEMS):
            if ratings[u][i] != None:
                r = (ratings[u][i] - MIN_RATING) / (MAX_RATING - MIN_RATING)

                for k in range(NUM_FACTORS):
                    gamma[u][k] += lmbda[u][i][k]

                    ep[i][k] += lmbda[u][i][k] * R * r
                    em[i][k] += lmbda[u][i][k] * R * (1 - r)

Iteración 1 de 5
Iteración 2 de 5
Iteración 3 de 5
Iteración 4 de 5
Iteración 5 de 5


Ahora podemos calcular los factores:

In [0]:
a = [[0 for _ in range(NUM_FACTORS)] for _ in range(NUM_USERS)] 

for u in range(NUM_USERS):
    summation = sum([_ for _ in gamma[u]])
    for k in range(NUM_FACTORS):
        a[u][k] = gamma[u][k] / summation

In [0]:
b = [[0 for _ in range(NUM_FACTORS)] for _ in range(NUM_ITEMS)]

for i in range(NUM_ITEMS):
    for k in range(NUM_FACTORS):
        b[i][k] = ep[i][k] / (ep[i][k] + em[i][k])

## Cálculo de las predicciones

Como hemos comentado, calcular la predicción del voto del usuario *u* al item *i* implicar realizar el producto escalar de sus vectores de factores. La siguiente función realiza esta operación:


In [0]:
def compute_prediction (a_u, b_i):
  prediction = 0
  for k in range(NUM_FACTORS):
    prediction += a_u[k] * b_i[k]
  return prediction * (MAX_RATING - MIN_RATING) + MIN_RATING

## Cálculo de las recomendaciones

El cálculo de las recomendaciones, por lo general, simplemente implica seleccionar los *N* items con una predicción más alta. Por ejemplo, si quisiéramos recomendar *N = 3* items a un usuario que tuviera las siguientes predicciones:

|   	| i1 	| i2 	| i3 	| i4 	| i5 	| i6 	| i7 	| i8 	| i9 	| i10 	|
|:-:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|-----	|
| u 	|   	|  2,9 	|    	|  4,7 	|  5,0 	|    	|  1,2 	|    	|   	|  3,1 	|

Se le recomendarían a dicho usuario los items *i5*, *i4* e *i10*.


##Cálculo del MAE

En esta sección vamos a mostrar cómo calcular el error medio absoluto (MAE) de las predicciones realizadas por el algoritmo BNMF.

Para ello, lo primero que debemos hacer es calcular las predicciones para todos los items que haya recibido una votación de test:

In [0]:
predictions = [[None for _ in range(NUM_ITEMS)] for _ in range(NUM_USERS)] 

for u in range(NUM_USERS):
  for i in range(NUM_ITEMS):
    if test_ratings[u][i] != None:
      predictions[u][i] = compute_prediction(a[u], b[i])

Y, a continuación, calculamos el MAE:

In [0]:
def get_user_mae (u):
  mae = 0
  count = 0
  
  for i in range(NUM_ITEMS):
    if test_ratings[u][i] != None and predictions[u][i] != None:
      mae += abs(test_ratings[u][i] - predictions[u][i])
      count += 1
  
  if count > 0:
    return mae / count
  else:
    return None

In [0]:
def get_mae ():
  mae = 0
  count = 0
  
  for u in range(NUM_USERS):
    user_mae = get_user_mae(u)
      
    if user_mae != None:
      mae += user_mae
      count += 1
  
  
  if count > 0:
    return mae / count
  else:
    return None   

In [21]:
mae = get_mae()
print("System MAE = " + str(mae))

System MAE = 0.8910716233437258


## Entendiendo los factores

En el modelo BNMF los factores se almacenan en las matrices $a$ y $b$. Como se ha comentado anteriormente, estas matrices tienen una interpretación probabilística. De este modo:

- $a_u$ representa una distribución de probabilidad que representa la probabilidad del usuario $u$ de pertenecer a cada cluster.

- $b_i$ representa la probabilidad de que a un usuario del cluster $k$ le interese el item $i$.

## Referencias

Hernando, A., Bobadilla, J., & Ortega, F. (2016). **A non negative matrix factorization for collaborative filtering recommender systems based on a Bayesian probabilistic model**. Knowledge-Based Systems, 97, 188-202.

---

*Este documento ha sido desarrollado por **Fernando Ortega**. Dpto. Sistemas Informáticos, ETSI de Sistemas Informáticos, Universidad Politécnica de Madrid.*

*Última actualización: Marzo de 2019*


<img src="https://drive.google.com/uc?export=view&id=1QuQDHyH_yrRbNt6sGzoZ8YcvFGEGlnWZ" alt="CC BY-NC">